In [ ]:
import streamlit as st
import pandas as pd
import sqlite3
from pathlib import Path
from typing import Optional


def _get_db_path() -> Path:
    """
    Return database Path resolved relative to the project root:
    <project_root>/data/daily_jobs.db
    """
    return Path(__file__).resolve().parents[1] / "data" / "daily_jobs.db"



def search_tags_box(limit: int = 1000) -> Optional[str]:
    """
    Search tags using four fields:
    - Object_Tag 
    - Father_Tag 
    - Unit_Code (exact)
    - Train (exact)
    
    Only executes when "Search" button is clicked.
    Returns selected Object_Tag from results.
    """
    with st.expander("🔍 Search Object Tag", expanded=True):
        # --- Four columns for search inputs ---
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            search_tag = st.text_input("Object Tag").strip()
        with col2:
            search_father = st.text_input("Father Tag").strip()
        with col3:
            search_unit = st.text_input("Unit").strip()
        with col4:
            search_train = st.text_input("Train").strip()

        # --- Search button ---
        search_clicked = st.button("🔎 Search")

        if not search_clicked:
            return None  # only search when button clicked

        db_path = _get_db_path()
        if not db_path.exists():
            st.error(f"❌ Database not found at: {db_path}")
            return None

        conn = None
        try:
            conn = sqlite3.connect(db_path, check_same_thread=False, timeout=10)

            # --- Build dynamic query ---
            conditions = []
            params = []

            if search_tag:
                conditions.append("Object_Tag LIKE ?")
                params.append(f"{search_tag}%")   # prefix search
            
            if search_father:
                conditions.append("Father_Tag LIKE ?")
                params.append(f"{search_father}%")  # prefix search
            
            if search_unit:
                conditions.append("Unit_Code = ?")
                params.append(search_unit)
            
            if search_train:
                conditions.append("Train = ?")
                params.append(search_train)

            
            if not conditions:
                st.warning("Enter at least one search criteria.")
                return None

            where_clause = " AND ".join(conditions)
            sql = f"SELECT * FROM objects WHERE {where_clause} ORDER BY Object_Tag LIMIT ?"
            params.append(limit)
            

            df = pd.read_sql(sql, conn, params=params)



        except Exception as e:
            st.error(f"Database query failed: {e}")
            return None

        finally:
            if conn:
                conn.close()

        if df.empty:
            st.warning("No records found for your search.")
            return None

        st.success(f"Found {len(df)} record(s).")
        st.dataframe(df, use_container_width=True)
        st.info("changed")

        # --- Select a tag from results ---
        selected_tag = st.selectbox("Select a tag to continue:", df["Object_Tag"].tolist())
        return selected_tag
